In [1]:
#installing extra libraries on prod
!pip install zeno-etl-libs-v3==1.0.1
!pip install openpyxl==3.0.9
!pip install nltk==3.6.7
!pip install tweepy==4.3.0
!pip install apiclient==1.0.4

In [2]:
"""
Fetching Playstore reviews on daily basis
Author : neha.karekar@zeno.health
"""

'\nFetching Playstore reviews on daily basis\nAuthor : neha.karekar@zeno.health\n'

In [3]:
import argparse
import sys
import re
import os
import pandas as pd
import dateutil
import datetime
from dateutil.tz import gettz
import numpy as np
import time
import datetime
import os
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords

In [4]:
nltk.download('stopwords')
nltk.download('punkt')  # divides a whole text data into sentences
nltk.download('vader_lexicon')
import tweepy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
sys.path.append('../../../..')

In [6]:
from zeno_etl_libs.logger import get_logger
from zeno_etl_libs.db.db import DB
from zeno_etl_libs.helper.aws.s3 import S3
from zeno_etl_libs.helper import helper
from zeno_etl_libs.helper.email.email import Email

## Pass Params


In [7]:
env = "dev"
full_run = 0
email_to ="neha.karekar@zeno.health"

In [8]:
os.environ['env'] = env
logger = get_logger()
logger.info(f"full_run: {full_run}")


2022-06-24 13:01:31,071 - root - INFO - full_run: 0


In [9]:
rs_db = DB(read_only=False)
rs_db.open_connection()
s3 = S3()

In [10]:
schema = 'prod2-generico'
table_name = 'zeno-tweets'
table_info = helper.get_table_info(db=rs_db, table_name=table_name, schema=schema)

In [11]:
# max of data
table_q = """
select
            max("tweet-created-at") max_exp
        from
            "prod2-generico"."zeno-tweets" 
        """
max_exp_date = rs_db.get_df(table_q)
max_exp_date['max_exp'].fillna(np.nan, inplace=True)
print(max_exp_date.info())
max_exp_date = max_exp_date['max_exp'].to_string(index=False)
print(max_exp_date)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   max_exp  1 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 136.0 bytes
None
2022-06-22 19:50:03


In [12]:
# params
if full_run or max_exp_date == 'NaN':
    start = '2017-05-13'
else:
    start = max_exp_date
start = dateutil.parser.parse(start)

In [13]:
# defining keys and tokens

consumer_key = 'c57SU7sulViKSmjsOTi4kTO3W'
consumer_secret = 'cNT3yk5ibQ315AWNCJHgE9ipCGlM1XnenHZu9cBWaVL3q7fPew'
access_token = '796747210159517701-DhOBQgwzeb6q4eXlI4WjwPRJH1CuEIT'
access_token_secret = 'sMrnPZ4ExI8um43wquUvFEUCTyY61HYRf7z3jv00ltXlt'

In [14]:
# making api connection

# authentication

def auth(consumer_key, consumer_secret, access_token, access_token_secret):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    return api


In [15]:
api = auth(consumer_key, consumer_secret, access_token, access_token_secret)

In [16]:
# remove url
def remove_url(txt):
    """Replace URLs found in a text string with nothing
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())


In [17]:
# searching for the keyword in tweeter and tokenizing it
def tweet(search_term, count=100000):
    # Create a custom search term and define the number of tweets
    tweets = api.search_tweets(search_term, count=count)

    # Remove URLs
    tweets_no_urls = [remove_url(tweet.text) for tweet in tweets]
    # lowercase
    tweet_data = [sent_tokenize(x.lower()) for x in tweets_no_urls]
    tweet_data = pd.DataFrame(data=tweet_data, columns=['tweetext'])
    tweet_att = [[search_term, x.lang, x.user.location, x.created_at, x.id, x.user.name,
                  x.user.followers_count, x.user.friends_count, x.text, x.place, x.user.time_zone] for x in tweets]
    tweet_att = pd.DataFrame(data=tweet_att, columns=['search_term', 'lang', 'loc', 'created-at', 'id', 'username',
                                                      'followers', 'friends', 'og tweet', 'place', 'Tz'])
    final_data = pd.concat([tweet_data, tweet_att], axis=1)
    return final_data


In [18]:
# removing stopwords
def remove_sw(sent, corpus):
    stop_words = set(stopwords.words(corpus))

    word_tokens = word_tokenize(sent)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    filtered_sentence = ' '.join(filtered_sentence)
    return [filtered_sentence]

In [19]:
# finding sentiment intensity analyzer
def sentiment_analyser(lst):
    sid = SentimentIntensityAnalyzer()
    sentiment = [sid.polarity_scores(x) for x in lst]
    neg = [sid.polarity_scores(x)['neg'] for x in lst]
    neu = [sid.polarity_scores(x)['neu'] for x in lst]
    pos = [sid.polarity_scores(x)['pos'] for x in lst]
    comp = [sid.polarity_scores(x)['compound'] for x in lst]

    return neg[0], neu[0], pos[0], comp[0]

In [20]:
# running all above functions
def run_all(search_term, count=1000000):
    print("API handshake successful")
    print("Searching for term ", search_term)
    tweet_data = tweet(search_term, count=count)
    # print(tweet_data)
    print("Removing stopwords")
    sw = 'english'
    if tweet_data.empty:
        return tweet_data
    else:
        tweet_data['tweetext_filter'] = tweet_data['tweetext'].apply(lambda x: remove_sw(x, sw), 1)
        print("Analysing sentiment for ", search_term)
        tweet_data['neg', 'neu', 'pos', 'comp'] = tweet_data['tweetext_filter'].apply(lambda x: sentiment_analyser(x), 1)
        tweet_data[['neg', 'neu', 'pos', 'comp']] = tweet_data['neg', 'neu', 'pos', 'comp'].apply(pd.Series)
        tweet_data.drop(columns=('neg', 'neu', 'pos', 'comp'), inplace=True)
        # sentiment, neg, neu, pos, comp = sentiment_analyser(tweets)
        # df = build_df(pos,neg,neu,comp, tweets)
        print('Done \n')
        return tweet_data

In [21]:
search_terms = ['#zeno_health','@zeno_health']

In [22]:
tws = pd.DataFrame()
try:
    for search_term in search_terms:
        tw = run_all(search_term, count=1000000)
        tws = pd.concat([tws, tw], axis=0)
        print('Done')
        tws = tws[((tws['lang'].isin(['en', 'hi']) & (~tws['tweetext'].str.startswith('rt'))))]
except BaseException as e:
    print('failed on_status,', str(e))
    time.sleep(3)
tws

API handshake successful
Searching for term  #zeno_health
Removing stopwords
Done
API handshake successful
Searching for term  @zeno_health
Removing stopwords
Analysing sentiment for  @zeno_health
Done 

Done


,tweetext,search_term,lang,loc,created-at,id,username,followers,friends,og tweet,place,Tz,tweetext_filter,neg,neu,pos,comp
0,pharmeasyapp,@zeno_health,hi,"Mumbai, India",2022-06-22 14:20:03+00:00,1539614195708477441,Jafar Usman Shaikh,77,503,@pharmeasyapp अभी तक हमारे कंप्लेंट एक्साइलेट ...,None,None,[pharmeasyapp],0.0,1.0,0.0,0.0
6,zenohealth a bearded cannot do the job of a ph...,@zeno_health,en,Uttar Pradesh/Jaipur/Mumbai,2022-06-21 07:45:25+00:00,1539152496907923456,DRx. Hamid Raza,7882,4409,"- @zeno_health ,\nA bearded cannot do the job ...",None,None,[zenohealth bearded job pharmacist companywhy],0.0,1.0,0.0,0.0
7,siddharthgadia1 not got my full report magnesi...,@zeno_health,en,bang lore,2022-06-20 15:45:04+00:00,1538910816153616384,Prasun Ashis Sengupta,37,447,"@SiddharthGadia1 Not got my full report , magn...",None,None,[siddharthgadia1 got full report magnesium rep...,0.0,1.0,0.0,0.0
8,zenohealth magnesium report not got still waiting,@zeno_health,en,bang lore,2022-06-20 15:42:58+00:00,1538910288111714304,Prasun Ashis Sengupta,37,447,@zeno_health Magnesium report not got still wa...,None,None,[zenohealth magnesium report got still waiting],0.0,1.0,0.0,0.0


In [23]:
if tws.empty:
    print('DataFrame is empty!')
    exit()

In [24]:
tws = tws[
    ['og tweet', 'id', 'created-at', 'search_term', 'lang', 'loc', 'username', 'followers', 'friends', 'neg', 'neu',
     'pos', 'comp']]
dict = {'id': 'tweet-id',
        'og tweet': 'tweet',
        'search_term': 'search-term',
        'lang': 'language',
        'loc': 'location',
        'created-at': 'tweet-created-at',
        'pos': 'positive-sentiment',
        'neu': 'neutral-sentiment',
        'neg': 'negative-sentiment',
        'comp': 'compound-sentiment'}
tws.rename(columns=dict, inplace=True)
tws['tweet-created-at'] = pd.to_datetime(tws['tweet-created-at']). \
    dt.tz_convert('Asia/Kolkata').dt.tz_localize(None)

c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
# etl
tws['created-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
tws['updated-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
tws['created-by'] = 'etl-automation'
tws['updated-by'] = 'etl-automation'

c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\lenovo\documents\generico-projects\etl\etl_env\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying

In [26]:
tws['tweet-type'] = np.where(tws['negative-sentiment'] >= tws['positive-sentiment'], 'Detractor', 'Promoter')
tws_mail = tws[['tweet-id', 'tweet', 'tweet-created-at', 'search-term', 'language', 'location', 'username', 'followers',
                'friends', 'tweet-type']]
tws_mail = tws_mail.sort_values(by=['tweet-type'], ascending=True)
tws_mail = tws_mail[(tws_mail['tweet-created-at'] > start)]
tws = tws[(tws['tweet-created-at'] > start)]

In [27]:
if tws.empty:
    print('DataFrame is empty!')
    rs_db.close_connection()
    exit()

DataFrame is empty!
Redshift DB connection closed successfully.


In [65]:
tws.columns = [c.replace('_', '-') for c in tws.columns]

In [66]:
if isinstance(table_info, type(None)):
    raise Exception(f"table: {table_name} do not exist, create the table first")
else:
    logger.info(f"Table:{table_name} exists")

2022-06-24 12:58:00,342 - root - INFO - Table:zeno-tweets exists
2022-06-24 12:58:00,342 - root - INFO - Table:zeno-tweets exists


In [67]:
truncate_query = f''' DELETE FROM "{schema}"."{table_name}" where "tweet-created-at" >'{start}' '''
print(truncate_query)
rs_db.execute(truncate_query)

 DELETE FROM "prod2-generico"."zeno-tweets" where "tweet-created-at" >'2017-05-13 00:00:00' 


In [68]:
s3.write_df_to_db(df=tws[table_info['column_name']], table_name=table_name, db=rs_db,
                  schema=schema)
file_name = 'Zeno_Tweets.xlsx'
file_path = s3.write_df_to_excel(data={'Zeno Tweets': tws_mail}, file_name=file_name)

S3 object(uri: s3://aws-glue-temporary-921939243643-ap-south-1/temp_1656055685080.csv) delete response: {'ResponseMetadata': {'RequestId': 'BEHB3C7JT6ZZHEQC', 'HostId': 'GjYrgimhUpH/1zTROddWFittBxwftqLpfZtPDfUT/NomkeHJNJ5emKFCh66Cs0W8vMG2L9CkQCE=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'GjYrgimhUpH/1zTROddWFittBxwftqLpfZtPDfUT/NomkeHJNJ5emKFCh66Cs0W8vMG2L9CkQCE=', 'x-amz-request-id': 'BEHB3C7JT6ZZHEQC', 'date': 'Fri, 24 Jun 2022 07:28:05 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}
S3 object(uri: s3://aws-glue-temporary-921939243643-ap-south-1/temp_1656055685080.csv) delete response: {'ResponseMetadata': {'RequestId': 'BEH25PZYHMC7CNN4', 'HostId': 'SYR26P0aRbfF9QX1VDHGJ7lgdRqe1G/rjXqTDfZegVgldYFVqJhYe08Ao448hMwvCOIBcbdNAVo=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'SYR26P0aRbfF9QX1VDHGJ7lgdRqe1G/rjXqTDfZegVgldYFVqJhYe08Ao448hMwvCOIBcbdNAVo=', 'x-amz-request-id': 'BEH25PZYHMC7CNN4', 'date': 'Fri, 24 Jun 2022 07:28:05 GMT', 'server': 'AmazonS3'}, 'RetryA

In [69]:
email = Email()
# file_path ='/Users/Lenovo/Downloads/utter.csv'
email.send_email_file(subject="Zeno Tweets",
                      mail_body='Zeno Tweets',
                      to_emails=email_to, file_uris=[], file_paths=[file_path])

2022-06-24 12:58:06,387 - root - INFO - Email sending successful: {"status":100,"message":"Successfully called back !","is_error":false,"data":"Email sending..."}
2022-06-24 12:58:06,387 - root - INFO - Email sending successful: {"status":100,"message":"Successfully called back !","is_error":false,"data":"Email sending..."}
2022-06-24 12:58:06,387 - root - INFO - Email sending successful: {"status":100,"message":"Successfully called back !","is_error":false,"data":"Email sending..."}


In [70]:
rs_db.close_connection()

Redshift DB connection closed successfully.
